# Imports

In [ ]:
# Standard libraries
import unicodedata
import hashlib
from collections import Counter
from pathlib import Path

# Data libraries
import pandas as pd
import numpy as np

# Optional: display options
pd.set_option('display.max_colwidth', 120)
pd.set_option('display.width', 120)

# Read CSVs

In [2]:
# Small robust CSV loader: try strict first, then fall back to tolerant read
def load_csv_robust(csv_path: Path, sep: str = ';') -> pd.DataFrame:
    print(f"Using: {csv_path}")
    try:
        df_local = pd.read_csv(csv_path, sep=sep)
        used_engine = 'c'
    except Exception as e:
        print(f"Strict read failed: {type(e).__name__}: {e}")
        df_local = pd.read_csv(csv_path, sep=sep, engine='python', on_bad_lines='skip')
        used_engine = 'python'
    print(f"Loaded {len(df_local):,} rows, {len(df_local.columns)} columns (engine={used_engine})")
    return df_local

# File paths (absolute to avoid working-directory issues)
base_root = Path(r"C:\Users\nikol\MT-repo")
base_log_path = base_root / 'data/201026 SEH Basis en Triage.csv'
orders_path = base_root / 'data/201026 SEH Orders.csv'

### Base Log 

In [3]:
# Read the main ED registrations/triage CSV
df = load_csv_robust(base_log_path, sep=';')
display(df.head(5))

Using: C:\Users\nikol\MT-repo\data\201026 SEH Basis en Triage.csv
Strict read failed: ParserError: Error tokenizing data. C error: Expected 29 fields in line 301, saw 30

Loaded 77,030 rows, 29 columns (engine=python)


,SEHRegistratienummer,Patientcode,Age,Gender,Specialismecode,RegitrationDate,RegitrationTime,ArrivelDate,ArrivelTime,TriageDate,...,Practice,Complaint,EMV,Pulse rate,Respiratoise rate,Oxygen level,Systolic/Diastolic,Temperature,Dismissal To,Admission Department
0,0000155008,61B9DFBE79E3BD2F45ADC7A502502D92,0.0,Man,KIN,2018-12-31 00:00:00.000,23:50,2019-01-01 00:00:00.000,10:05,2019-01-01 00:00:00.000,...,Overige,Herbeoordeling koorts,NaN,NaN,NaN,NaN,NaN,NaN,Naar huis - controle poli,NaN
1,0000155011,914C433AF42956364DABDA127C4C67CA,26.0,Man,CHI,2019-01-01 00:00:00.000,00:27,2019-01-01 00:00:00.000,00:35,2019-01-01 00:00:00.000,...,Overige,Intox en val op achterhoofd,15.0,77.0,14.0,100.0,112/60,36.0,Opname,10B
2,0000155012,B894848BF21136EED56A12362A1527CE,11.0,Vrouw,CHI,2019-01-01 00:00:00.000,00:29,2019-01-01 00:00:00.000,00:29,2019-01-01 00:00:00.000,...,Overige,vuurwerk tegen gelaat gekregen,NaN,NaN,NaN,NaN,NaN,NaN,Naar huis - geen vervolgbeh.,NaN
3,0000155013,5D988FF7BA7267FF1CDA35B5ED7252B0,73.0,Man,NEU,2019-01-01 00:00:00.000,00:29,2019-01-01 00:00:00.000,00:37,2019-01-01 00:00:00.000,...,Overige,CVA,NaN,91.0,16.0,98.0,155/99,36.5,Opname,AODA
4,0000155016,086D9129697F6A11B6DB8736C983C84A,11.0,Man,CHI,2019-01-01 00:00:00.000,01:03,2019-01-01 00:00:00.000,01:03,2019-01-01 00:00:00.000,...,Overige,brandwonden,NaN,NaN,NaN,NaN,NaN,NaN,Naar huis - controle poli,NaN


### Orders log

In [4]:
# Read the Orders CSV
orders_df = load_csv_robust(orders_path, sep=';')
display(orders_df.head(5))

Using: C:\Users\nikol\MT-repo\data\201026 SEH Orders.csv
Strict read failed: ParserError: Error tokenizing data. C error: Expected 5 fields in line 5896, saw 6

Loaded 1,148,302 rows, 5 columns (engine=python)


,SEH_KoppelID,Specialismecode,SEH_orders,Order_Startdate,order_starttime
0,0000155012,CHI,Medicatieverificatie SEH,2019-01-01 00:00:00.000,00:30
1,0000155011,CHI,Medicatieverificatie SEH,2019-01-01 00:00:00.000,00:35
2,0000155013,NEU,Radiologie,2019-01-01 00:00:00.000,00:37
3,0000155013,NEU,Medicatieverificatie SEH,2019-01-01 00:00:00.000,00:37
4,0000155012,CHI,Pijnscore herhalen,2019-01-01 00:00:00.000,00:37


### Filter by Specialismecode (right after import)
Reduce the dataset early by keeping only rows whose `Specialismecode` is in a provided list.
- Applies to both the base log (`df`) and the Orders log (`orders_df`).
- Set the list of codes in `SPECIALISMECODES` and run this cell.
- If the list is empty, no filtering is applied.

In [5]:
# List of Specialismecodes to keep (exact matches, case-insensitive)
# Example: SPECIALISMECODES = ['SEH', 'CARD', 'INT']
SPECIALISMECODES = ["ORT"]

# Normalize to strings for matching
_keep = [str(x) for x in SPECIALISMECODES]

# Base log filter
_before_rows = len(df)
if _keep:
    df = df[df['Specialismecode'].astype(str).str.upper().isin([c.upper() for c in _keep])].copy()
_after_rows = len(df)
print(f"[base] Specialismecode filter: rows {_after_rows}/{_before_rows} kept" + (" (no filter list provided)" if not _keep else ""))

# Orders filter
_before_rows_o = len(orders_df)
if _keep:
    orders_df = orders_df[orders_df['Specialismecode'].astype(str).str.upper().isin([c.upper() for c in _keep])].copy()
_after_rows_o = len(orders_df)
print(f"[orders] Specialismecode filter: rows {_after_rows_o}/{_before_rows_o} kept" + (" (no filter list provided)" if not _keep else ""))

# Quick peek after filtering
display(df.head(3))
display(orders_df.head(3))

[base] Specialismecode filter: rows 2518/77030 kept
[orders] Specialismecode filter: rows 28415/1148302 kept


,SEHRegistratienummer,Patientcode,Age,Gender,Specialismecode,RegitrationDate,RegitrationTime,ArrivelDate,ArrivelTime,TriageDate,...,Practice,Complaint,EMV,Pulse rate,Respiratoise rate,Oxygen level,Systolic/Diastolic,Temperature,Dismissal To,Admission Department
59,0000155083,67A29F6B254D19FF45E810A5C96AF912,86.0,Vrouw,ORT,2019-01-01 00:00:00.000,10:23,2019-01-01 00:00:00.000,10:45,2019-01-01 00:00:00.000,...,Huisarts,pols #?,NaN,NaN,NaN,NaN,NaN,NaN,Naar huis - controle poli,NaN
60,0000155084,78FC22DC87AFC8E7A85749D3D04F7EFF,92.0,Vrouw,ORT,2019-01-01 00:00:00.000,10:50,2019-01-01 00:00:00.000,12:38,2019-01-01 00:00:00.000,...,Huisarts,verdenking heup#,NaN,76.0,NaN,97.0,160/82,36.6,Opname,10B
70,0000155094,DECA2109267F16D9123E771F337375C7,90.0,Vrouw,ORT,2019-01-01 00:00:00.000,12:27,2019-01-01 00:00:00.000,12:36,2019-01-01 00:00:00.000,...,Overige,Collum#,NaN,99.0,NaN,99.0,98/40,37.0,Opname,10B


,SEH_KoppelID,Specialismecode,SEH_orders,Order_Startdate,order_starttime
707,0000155083,ORT,Medicatieverificatie SEH,2019-01-01 00:00:00.000,10:45
782,0000155083,ORT,Beleidsorder SEH,2019-01-01 00:00:00.000,11:37
783,0000155083,ORT,Radiologieaanvraag SEH,2019-01-01 00:00:00.000,11:37


# Building event log

In [6]:
# Remove invalid cases with Age == 0 (if column exists)
if 'Age' in df.columns:
    age_num = pd.to_numeric(df['Age'], errors='coerce')
    mask_zero = age_num == 0
    n_drop = int(mask_zero.sum())
    if n_drop > 0:
        df = df.loc[~mask_zero].copy()
    print(f"Dropped {n_drop} row(s) with Age == 0.")
else:
    print("Column 'Age' not found; no rows dropped for Age == 0.")

Dropped 10 row(s) with Age == 0.


### Filter case attributes
- Keep only a small, focused set of case-level attributes plus the event date/time columns needed for parsing.
- Edit the `keep_case_attrs` list below to include the case attributes you want to retain.

In [7]:
# Choose and keep a focused subset of case attributes
case_id_col = "SEHRegistratienummer"

# Event columns needed later to construct per-activity timestamps
raw_event_cols = [
    "RegitrationDate","RegitrationTime",
    "ArrivelDate","ArrivelTime",
    "TriageDate","TriageTime",
    "Start Treatment Date","Start TreatmentTime",
    "DepartDate","DepartTime",
]

# NOTE: Edit this list to include the case attributes you'd like to retain (if present)
keep_case_attrs = [
    case_id_col,
    "Age",
    "Gender",
    "Specialismecode",
]

# Compute which columns are present and keep them, alongside event columns
present_attrs = [c for c in keep_case_attrs if c in df.columns]
present_event_cols = [c for c in raw_event_cols if c in df.columns]
cols_to_keep = list(dict.fromkeys([case_id_col] + present_attrs + present_event_cols))  # preserve order, unique

_before_cols = len(df.columns)
df = df.loc[:, [c for c in cols_to_keep if c in df.columns]].copy()

print(f"Filtered columns: kept {len(df.columns)} of {_before_cols}.")
print("Kept columns:", df.columns.tolist())

display(df.head(5))

Filtered columns: kept 14 of 29.
Kept columns: ['SEHRegistratienummer', 'Age', 'Gender', 'Specialismecode', 'RegitrationDate', 'RegitrationTime', 'ArrivelDate', 'ArrivelTime', 'TriageDate', 'TriageTime', 'Start Treatment Date', 'Start TreatmentTime', 'DepartDate', 'DepartTime']


,SEHRegistratienummer,Age,Gender,Specialismecode,RegitrationDate,RegitrationTime,ArrivelDate,ArrivelTime,TriageDate,TriageTime,Start Treatment Date,Start TreatmentTime,DepartDate,DepartTime
59,0000155083,86.0,Vrouw,ORT,2019-01-01 00:00:00.000,10:23,2019-01-01 00:00:00.000,10:45,2019-01-01 00:00:00.000,10:51,2019-01-01 00:00:00.000,10:45,2019-01-01 00:00:00.000,14:20
60,0000155084,92.0,Vrouw,ORT,2019-01-01 00:00:00.000,10:50,2019-01-01 00:00:00.000,12:38,2019-01-01 00:00:00.000,12:49,2019-01-01 00:00:00.000,12:38,2019-01-01 00:00:00.000,18:50
70,0000155094,90.0,Vrouw,ORT,2019-01-01 00:00:00.000,12:27,2019-01-01 00:00:00.000,12:36,2019-01-01 00:00:00.000,12:50,2019-01-01 00:00:00.000,12:36,2019-01-01 00:00:00.000,16:00
80,0000155105,47.0,Man,ORT,2019-01-01 00:00:00.000,14:02,2019-01-01 00:00:00.000,14:02,2019-01-01 00:00:00.000,14:29,2019-01-01 00:00:00.000,14:26,2019-01-01 00:00:00.000,15:59
84,0000155109,26.0,Man,ORT,2019-01-01 00:00:00.000,14:25,2019-01-01 00:00:00.000,14:25,2019-01-01 00:00:00.000,14:41,2019-01-01 00:00:00.000,14:40,2019-01-01 00:00:00.000,16:49


### Event parsing
- For each event (registration, arrivel, triage, start treatment, depart):
  - If both date and time are missing, omit that event for that case (do not add an event row).
  - If exactly one of date/time is missing, drop the entire case (partial timestamp).
  - If both are present but the combined timestamp is invalid, drop the entire case.
- After enforcing the rules above, construct one row per valid event with a parsed timestamp and attach case attributes.

In [8]:
# Parse events, drop cases on partial/invalid, and build event frames (robust date parsing + diagnostics)

# Define event timestamp columns (as provided in the raw data)
events = [
    {"name": "registration", "date": "RegitrationDate", "time": "RegitrationTime"},
    {"name": "arrivel",       "date": "ArrivelDate",       "time": "ArrivelTime"},
    {"name": "triage",        "date": "TriageDate",        "time": "TriageTime"},
    {"name": "start treatment","date": "Start Treatment Date","time": "Start TreatmentTime"},
    {"name": "depart",        "date": "DepartDate",        "time": "DepartTime"},
]

# Columns used only to build event timestamps (will be dropped later)
raw_event_cols = [
    "RegitrationDate","RegitrationTime",
    "ArrivelDate","ArrivelTime",
    "TriageDate","TriageTime",
    "Start Treatment Date","Start TreatmentTime",
    "DepartDate","DepartTime",
]

# Helpers

def _is_empty(series: pd.Series) -> pd.Series:
    return series.astype(str).str.strip().str.lower().isin(["", "nan", "nat", "null", "none"])


def _normalize_date_to_ymd(series: pd.Series) -> pd.Series:
    """Parse any parseable date-like string (even if it contains time) and return YYYY-MM-DD strings.
    Unparseable entries become NA (to be handled by caller)."""
    dt = pd.to_datetime(series, errors="coerce", yearfirst=True)
    return dt.dt.strftime("%Y-%m-%d")


def _normalize_time(series: pd.Series) -> pd.Series:
    """Lightly normalize time strings: replace '.' with ':', convert HHMM -> HH:MM; keep others as-is."""
    s = series.astype(str).str.strip()
    s = s.str.replace(".", ":", regex=False)
    # If exactly 4 digits (HHMM), convert to HH:MM
    s = s.str.replace(r"^(\d{2})(\d{2})$", r"\1:\2", regex=True)
    return s

# 1) Enforce drop rules: partial (one missing) or invalid parse -> drop entire case.
#    If both missing, we simply omit that event later (do not drop case).

cases_to_drop = set()
for spec in events:
    dcol, tcol = spec.get("date"), spec.get("time")

    # Only enforce row-level rules when both columns exist
    if dcol in df.columns and tcol in df.columns:
        date_s = df[dcol]
        time_s = df[tcol]
        date_empty = _is_empty(date_s)
        time_empty = _is_empty(time_s)
        both_empty = date_empty & time_empty
        partial_mask = date_empty ^ time_empty

        # Diagnostics: empties and partials
        n_date_empty = int(date_empty.sum())
        n_time_empty = int(time_empty.sum())
        n_both_empty = int(both_empty.sum())
        n_partial = int(partial_mask.sum())

        # Partial -> drop cases
        if n_partial > 0:
            dropped_cases = df.loc[partial_mask, case_id_col].dropna().unique().tolist()
            cases_to_drop.update(dropped_cases)

        # Both present -> parse normalized date + time; invalid -> drop cases
        both_present = (~date_empty) & (~time_empty)
        if both_present.any():
            date_norm = _normalize_date_to_ymd(date_s)
            time_norm = _normalize_time(time_s)
            invalid_date = date_norm.isna()
            dt_str = (date_norm.fillna("") + " " + time_norm.fillna("")).str.strip()
            ts = pd.to_datetime(dt_str.replace({"": pd.NA}), errors="coerce", yearfirst=True)
            invalid_combined = ts.isna()
            invalid_mask = both_present & (invalid_date | invalid_combined)

            # Diagnostics: invalids when both present
            n_both_present = int(both_present.sum())
            n_invalid_rows = int(invalid_mask.sum())
            cases_invalid = int(df.loc[invalid_mask, case_id_col].dropna().nunique())
            print(
                f"[{spec['name']}] both_present rows={n_both_present}, invalid_combined rows={n_invalid_rows} (cases={cases_invalid})"
            )

            if n_invalid_rows > 0:
                invalid_cases = df.loc[invalid_mask, case_id_col].dropna().unique().tolist()
                cases_to_drop.update(invalid_cases)
    else:
        # Columns not both present: treat as if event is globally missing -> do nothing, will be omitted
        missing = [c for c in [dcol, tcol] if c not in df.columns]
        print(f"[{spec['name']}] skipping drop checks (missing column(s): {missing}))")

n_cases_before = df[case_id_col].nunique()
if cases_to_drop:
    df = df[~df[case_id_col].isin(cases_to_drop)].copy()
    print(
        f"Dropped {len(cases_to_drop)} case(s) for partial/invalid timestamps. Remaining cases: {df[case_id_col].nunique()} (from {n_cases_before})."
    )
else:
    print("No cases dropped for partial/invalid timestamps across events.")

# 2) Build per-activity event frames from the filtered df — keep only valid timestamps

log_frames = []
trace_attr_cols = [c for c in df.columns if c != case_id_col and c not in raw_event_cols]

for spec in events:
    dcol, tcol = spec.get("date"), spec.get("time")

    # Must have both columns to construct an event
    if not (dcol in df.columns and tcol in df.columns):
        print(f"[{spec['name']}] no event built (missing date/time column)")
        continue

    date_s = df[dcol]
    time_s = df[tcol]
    date_empty = _is_empty(date_s)
    time_empty = _is_empty(time_s)

    # Only rows where both present
    both_present = (~date_empty) & (~time_empty)
    if not both_present.any():
        print(f"[{spec['name']}] no event rows (no rows with both date and time present)")
        continue

    date_norm = _normalize_date_to_ymd(date_s)
    time_norm = _normalize_time(time_s)

    # Build combined string and parse
    dt_str = (date_norm.fillna("") + " " + time_norm.fillna("")).str.strip()
    ts = pd.to_datetime(dt_str.replace({"": pd.NA}), errors="coerce", yearfirst=True)

    valid_mask = both_present & date_norm.notna() & ts.notna()
    n_kept = int(valid_mask.sum())
    if n_kept == 0:
        print(f"[{spec['name']}] no valid timestamps after normalization/parsing")
        continue

    ev_df = pd.DataFrame({
        case_id_col: df.loc[valid_mask, case_id_col],
        "activity": spec["name"],
        "timestamp": ts[valid_mask],
    })

    # Attach trace attributes (repeat across events), aligning with valid rows
    ev_df = pd.concat([ev_df, df.loc[valid_mask, trace_attr_cols]], axis=1)

    print(f"[{spec['name']}] kept {n_kept} event row(s)")
    log_frames.append(ev_df)

[registration] both_present rows=2508, invalid_combined rows=0 (cases=0)
[arrivel] both_present rows=2508, invalid_combined rows=0 (cases=0)
[triage] both_present rows=2457, invalid_combined rows=0 (cases=0)
[start treatment] both_present rows=2471, invalid_combined rows=0 (cases=0)
[depart] both_present rows=2508, invalid_combined rows=0 (cases=0)
Dropped 83 case(s) for partial/invalid timestamps. Remaining cases: 2425 (from 2508).
[registration] kept 2425 event row(s)
[arrivel] kept 2425 event row(s)
[triage] kept 2425 event row(s)
[start treatment] kept 2425 event row(s)
[depart] kept 2425 event row(s)


### Sorting and cleanup
- Sort events within each case by timestamp (ascending).
- If timestamps are equal, break ties using the default activity order:
  registration < arrivel < triage < start treatment < depart.
- Drop any index-like columns (e.g., Unnamed, index, idx).

In [9]:
# Concatenate frames and sort with explicit precedence for ties
if not log_frames:
    raise RuntimeError("No events could be constructed; check exact column names for date/time fields.")

log_df = pd.concat(log_frames, axis=0, ignore_index=True)

activity_order = {
    "registration": 0,
    "arrivel": 1,
    "triage": 2,
    "start treatment": 3,
    "depart": 4,
}
log_df["_act_order"] = log_df["activity"].map(activity_order).fillna(99)
log_df.sort_values(by=[case_id_col, "timestamp", "_act_order", "activity"], inplace=True)
log_df.drop(columns=["_act_order"], inplace=True)

# Cleanup: drop index-like columns if any propagated
idx_like_cols = [c for c in log_df.columns if str(c).lower().startswith("unnamed") or str(c).lower() in ("index", "idx")]
if idx_like_cols:
    print(f"Dropping index-like columns from event log: {idx_like_cols}")
    log_df.drop(columns=idx_like_cols, inplace=True)
    
log_df.reset_index(drop=True, inplace=True)

# Preview
display(log_df.head(10))

,SEHRegistratienummer,activity,timestamp,Age,Gender,Specialismecode
0,0000155083,registration,2019-01-01 10:23:00,86.0,Vrouw,ORT
1,0000155083,arrivel,2019-01-01 10:45:00,86.0,Vrouw,ORT
2,0000155083,start treatment,2019-01-01 10:45:00,86.0,Vrouw,ORT
3,0000155083,triage,2019-01-01 10:51:00,86.0,Vrouw,ORT
4,0000155083,depart,2019-01-01 14:20:00,86.0,Vrouw,ORT
5,0000155084,registration,2019-01-01 10:50:00,92.0,Vrouw,ORT
6,0000155084,arrivel,2019-01-01 12:38:00,92.0,Vrouw,ORT
7,0000155084,start treatment,2019-01-01 12:38:00,92.0,Vrouw,ORT
8,0000155084,triage,2019-01-01 12:49:00,92.0,Vrouw,ORT
9,0000155084,depart,2019-01-01 18:50:00,92.0,Vrouw,ORT


### Variants
- Compute per-case activity sequences from the current event log.
- Count variant frequencies and show the top ones with percentages.

In [10]:
# Helpers for variant analysis (reused)

def _compress_consecutive(seq):
    out = []
    last = object()
    for x in seq:
        if x != last:
            out.append(x)
            last = x
    return out


def compute_variant_counts(log_df, case_id_col, activity_col='activity', activity_tie_order=None, compress=True):
    """
    Returns variant_counts DataFrame with columns: ['variant', 'cases'] and a Series seq_per_case.
    - Sorts by (case, timestamp, tie-order, activity) if tie-order provided.
    - Optionally compresses consecutive duplicate activities within a case before building sequences.
    """
    df = log_df
    if activity_tie_order is not None:
        df = (df.assign(_act_order=df[activity_col].str.lower().map(activity_tie_order).fillna(99))
                .sort_values([case_id_col, 'timestamp', '_act_order', activity_col])
                .drop(columns=['_act_order']))
    else:
        df = df.sort_values([case_id_col, 'timestamp', activity_col])

    if compress:
        seq_per_case = df.groupby(case_id_col, sort=False)[activity_col] \
            .apply(lambda s: ' > '.join(_compress_consecutive(s.tolist())))
    else:
        seq_per_case = df.groupby(case_id_col, sort=False)[activity_col] \
            .apply(lambda s: ' > '.join(s.tolist()))

    variant_counts = (seq_per_case.value_counts()
                                  .rename_axis('variant')
                                  .reset_index(name='cases'))
    return variant_counts, seq_per_case

In [11]:
# Variants: compute per-case sequences and frequencies using shared helper

variant_counts, seq_per_case = compute_variant_counts(
    log_df,
    case_id_col=case_id_col,
    activity_col='activity',
    activity_tie_order={
        'registration': 0,
        'arrivel': 1,
        'triage': 2,
        'start treatment': 3,
        'depart': 4,
    },
    compress=True,
)

total_cases = int(seq_per_case.shape[0])
variant_counts['percent_of_cases'] = (variant_counts['cases'] / max(1, total_cases) * 100).round(2)

print(f"Total cases: {total_cases}")
print(f"Total unique variants: {len(variant_counts)}")
TOP_N = 20
print(f"Top {min(TOP_N, len(variant_counts))} variants by cases (with coverage totals):")

# Prepare top-N with a totals row
_top_n = min(TOP_N, len(variant_counts))
top_df = variant_counts.head(_top_n).copy()
coverage_cases = int(top_df['cases'].sum())
coverage_pct = round(coverage_cases / max(1, total_cases) * 100, 2)
totals_row = pd.DataFrame([
    {
        'variant': f'TOTAL (top {_top_n})',
        'cases': coverage_cases,
        'percent_of_cases': coverage_pct,
    }
])

display(pd.concat([top_df, totals_row], ignore_index=True))

Total cases: 2425
Total unique variants: 4
Top 4 variants by cases (with coverage totals):


,variant,cases,percent_of_cases
0,registration > arrivel > start treatment > triage > depart,2203,90.85
1,registration > arrivel > triage > start treatment > depart,218,8.99
2,registration > arrivel > start treatment > depart > triage,3,0.12
3,registration > triage > arrivel > start treatment > depart,1,0.04
4,TOTAL (top 4),2425,100.00


# Adding orders log
- Use the Orders data to construct additional event rows per case.
- Normalize order date/time like the base log: date → YYYY-MM-DD; time: replace '.' with ':' and convert HHMM → HH:MM.
- Omit rows where both date and time are missing; drop rows with partial or invalid timestamps.
- First, display a frequency table of order activities and optionally filter by minimum occurrences (parameters in the next code cell).
- Then, merge the prefiltered order events into the current event log and re-sort with tie-breakers (orders between triage and start treatment).

In [12]:
# Drop rare variants before integrating orders
# Keep only cases whose pre-orders variant appears in at least this many cases
MIN_VARIANT_CASES = 4

# Build per-case sequences using current row order (pre-orders log_df)
seq_per_case = log_df.groupby(case_id_col, sort=False)['activity'].apply(lambda s: ' > '.join(s.tolist()))
variant_counts = (seq_per_case.value_counts()
                               .rename_axis('variant')
                               .reset_index(name='cases'))

# Determine which variants to drop
variants_to_drop = set(variant_counts.loc[variant_counts['cases'] < int(MIN_VARIANT_CASES), 'variant'].tolist())

print(f"Pre-orders variants total: {len(variant_counts)}")
print(f"Variants to drop (< {MIN_VARIANT_CASES} cases): {len(variants_to_drop)}")

# Map case -> variant and filter cases
case_to_variant = seq_per_case
allowed_cases = set(case_to_variant[~case_to_variant.isin(variants_to_drop)].index)

before_cases = int(log_df[case_id_col].nunique())
before_events = int(len(log_df))

filtered_df = log_df[log_df[case_id_col].isin(allowed_cases)].copy()
kept_cases = int(filtered_df[case_id_col].nunique())
kept_events = int(len(filtered_df))

print(
    f"Kept {kept_cases}/{before_cases} cases "
    f"({(kept_cases/before_cases*100 if before_cases else 0):.2f}%), "
    f"{kept_events}/{before_events} events "
    f"({(kept_events/before_events*100 if before_events else 0):.2f}%)."
)

# Update global log
log_df = filtered_df

display(variant_counts.tail(5))  # show the bottom variants for reference
print("log_df has been filtered by pre-orders variant frequency.")

Pre-orders variants total: 4
Variants to drop (< 4 cases): 2
Kept 2421/2425 cases (99.84%), 12105/12125 events (99.84%).


,variant,cases
0,registration > arrivel > start treatment > triage > depart,2203
1,registration > arrivel > triage > start treatment > depart,218
2,registration > arrivel > start treatment > depart > triage,3
3,registration > triage > arrivel > start treatment > depart,1


log_df has been filtered by pre-orders variant frequency.


In [13]:
# Orders activity frequency (before filtering and integration)
# Build a valid orders subset consistent with base parsing and show activity counts.
# Keep only the TOP-K most frequent order activities for integration.

# Parameter: number of top activities to keep (by frequency)
ORDERS_TOP_K_ACTIVITIES = 30  # change as needed

# Column names in Orders
orders_case_col = 'SEH_KoppelID'
orders_date_col = 'Order_Startdate'
orders_time_col = 'order_starttime'
orders_name_col = 'SEH_orders_clean' if 'SEH_orders_clean' in orders_df.columns else ('SEH_orders' if 'SEH_orders' in orders_df.columns else None)
if orders_name_col is None:
    orders_df = orders_df.copy()
    orders_name_col = '_order_name_'
    orders_df[orders_name_col] = 'ORDER'

# Build masks using the same helpers as the base parsing
_date_s = orders_df[orders_date_col]
_time_s = orders_df[orders_time_col]
_date_empty = _is_empty(_date_s)
_time_empty = _is_empty(_time_s)

_both_present = (~_date_empty) & (~_time_empty)
_date_norm = _normalize_date_to_ymd(_date_s)
_time_norm = _normalize_time(_time_s)
_dt_str = (_date_norm.fillna('') + ' ' + _time_norm.fillna('')).str.strip()
_ts = pd.to_datetime(_dt_str.replace({'': pd.NA}), errors='coerce', yearfirst=True)
_invalid = _both_present & (_date_norm.isna() | _ts.isna())

# Keep only valid orders with case ids present in the base log
_valid = _both_present & (~_invalid) & orders_df[orders_case_col].isin(set(log_df[case_id_col].dropna().unique()))
kept = int(_valid.sum())
print(f"[orders] candidate valid rows for frequency table: {kept}")

if kept == 0:
    print("No valid orders available for frequency table.")
    # Provide an empty placeholder for downstream merge to consume safely
    orders_events_filtered = pd.DataFrame(columns=[case_id_col, 'activity', 'timestamp'])
else:
    orders_events = pd.DataFrame({
        case_id_col: orders_df.loc[_valid, orders_case_col],
        'activity': orders_df.loc[_valid, orders_name_col].astype(str).str.strip(),
        'timestamp': _ts[_valid],
    })
    if 'Specialismecode' in orders_df.columns and 'Specialismecode' not in orders_events.columns:
        orders_events['Specialismecode'] = orders_df.loc[_valid, 'Specialismecode']

    # Frequency BEFORE filtering
    orders_counts_pre = (
        orders_events['activity']
            .value_counts()
            .rename_axis('activity')
            .reset_index(name='events')
    )
    print("\nOrders activity frequency (before TOP-K filtering):")
    print(f"- Unique order activities: {orders_counts_pre.shape[0]}")
    display(orders_counts_pre)

    # Keep only the top-K activities
    k = int(max(0, ORDERS_TOP_K_ACTIVITIES))
    if k == 0:
        print("\nORDERS_TOP_K_ACTIVITIES is 0 → keeping no order activities.")
        orders_events_filtered = orders_events.iloc[0:0].copy()
    else:
        top_activities = orders_counts_pre['activity'].head(k).tolist()
        before_events = int(len(orders_events))
        before_acts = int(orders_counts_pre.shape[0])

        orders_events_filtered = orders_events[orders_events['activity'].isin(top_activities)].copy()
        after_events = int(len(orders_events_filtered))
        after_acts = int(orders_events_filtered['activity'].nunique())

        print(f"\nApplied TOP-K filter: keeping top {k} activities by frequency.")
        print(f"- Activities: {after_acts}/{before_acts} retained; Events: {after_events}/{before_events} retained")

        if after_events > 0:
            orders_counts_post = (
                orders_events_filtered['activity']
                    .value_counts()
                    .rename_axis('activity')
                    .reset_index(name='events')
            )
            print("\nOrders activity frequency (after TOP-K filtering):")
            display(orders_counts_post)
        else:
            print("No order events remain after TOP-K filtering.")

    print(f"\nPrepared 'orders_events_filtered' with {len(orders_events_filtered)} row(s) for integration.")

[orders] candidate valid rows for frequency table: 27975

Orders activity frequency (before TOP-K filtering):
- Unique order activities: 995


,activity,events
0,Pijnscore herhalen,3813
1,Beleidsorder SEH,2312
2,Radiologie,2018
3,Radiologieaanvraag SEH,1975
4,Externe verslaglegging,1835
...,...,...
990,cave delier,1
991,ivm DM diabetes verpleegkundige in consult,1
992,volledige hulp op bed,1
993,Vrij dieet tot nader order,1



Applied TOP-K filter: keeping top 30 activities by frequency.
- Activities: 30/995 retained; Events: 22381/27975 retained

Orders activity frequency (after TOP-K filtering):


,activity,events
0,Pijnscore herhalen,3813
1,Beleidsorder SEH,2312
2,Radiologie,2018
3,Radiologieaanvraag SEH,1975
4,Externe verslaglegging,1835
5,SEH Radiologie bespreking,1688
6,Beleidsorder SEH-VPL,1166
7,Ontslagbeleid Poliklinisch - SEH,1117
8,Afspraak maken (over 1 week),760
9,Medicatieverificatie SEH,747



Prepared 'orders_events_filtered' with 22381 row(s) for integration.


In [14]:
# Parse Orders timestamps like the base log, then combine into log_df

# This cell merges prefiltered orders (from the previous cell) into the event log.
# If the previous cell wasn't run, we'll fall back to computing valid orders without filtering.

# Column names in Orders
orders_case_col = 'SEH_KoppelID'
orders_date_col = 'Order_Startdate'
orders_time_col = 'order_starttime'
orders_name_col = 'SEH_orders_clean' if 'SEH_orders_clean' in orders_df.columns else ('SEH_orders' if 'SEH_orders' in orders_df.columns else None)
if orders_name_col is None:
    orders_df = orders_df.copy()
    orders_name_col = '_order_name_'
    orders_df[orders_name_col] = 'ORDER'

# Use prefiltered orders if available, otherwise compute valid orders quickly
if 'orders_events_filtered' in globals():
    orders_events = orders_events_filtered.copy()
    print(f"[orders] using prefiltered order events: {len(orders_events)} row(s)")
else:
    # Build masks using the same helpers as the base parsing
    _date_s = orders_df[orders_date_col]
    _time_s = orders_df[orders_time_col]
    _date_empty = _is_empty(_date_s)
    _time_empty = _is_empty(_time_s)
    _both_present = (~_date_empty) & (~_time_empty)
    _date_norm = _normalize_date_to_ymd(_date_s)
    _time_norm = _normalize_time(_time_s)
    _dt_str = (_date_norm.fillna('') + ' ' + _time_norm.fillna('')).str.strip()
    _ts = pd.to_datetime(_dt_str.replace({'': pd.NA}), errors='coerce', yearfirst=True)
    _invalid = _both_present & (_date_norm.isna() | _ts.isna())

    _valid = _both_present & (~_invalid) & orders_df[orders_case_col].isin(set(log_df[case_id_col].dropna().unique()))
    kept = int(_valid.sum())
    print(f"[orders] kept {kept} valid order event row(s) (no prefiltered set found)")

    orders_events = pd.DataFrame({
        case_id_col: orders_df.loc[_valid, orders_case_col],
        'activity': orders_df.loc[_valid, orders_name_col].astype(str).str.strip(),
        'timestamp': _ts[_valid],
    })
    if 'Specialismecode' in orders_df.columns and 'Specialismecode' not in orders_events.columns:
        orders_events['Specialismecode'] = orders_df.loc[_valid, 'Specialismecode']

# If nothing to merge, show the base log and exit
if orders_events.empty:
    print("No orders to integrate; keeping base event log unchanged.")
    display(log_df.head(10))
else:
    # Combine and sort with precedence: orders between triage and start treatment
    combined = pd.concat([log_df.assign(_is_order=False), orders_events.assign(_is_order=True)], ignore_index=True, sort=False)
    _precedence = {'registration': 0, 'arrivel': 1, 'triage': 2, 'start treatment': 4, 'depart': 5}
    combined['_order_rank'] = combined.apply(lambda r: 3 if bool(r.get('_is_order', False)) else _precedence.get(str(r.get('activity')).lower(), 99), axis=1)
    combined.sort_values(by=[case_id_col, 'timestamp', '_order_rank', 'activity'], inplace=True)
    combined.drop(columns=['_order_rank', '_is_order'], inplace=True, errors='ignore')
    combined.reset_index(drop=True, inplace=True)

    # Deduplicate exact duplicate events (same case, activity, timestamp)
    _dup_mask = combined.duplicated(subset=[case_id_col, 'activity', 'timestamp'], keep='first')
    _num_dups = int(_dup_mask.sum())
    if _num_dups > 0:
        print(f"Removed {_num_dups} exact duplicate event row(s) (same case, activity, timestamp).")
        combined = combined[~_dup_mask].copy()

    # Fill NaNs in attributes from the registration event of the same case id
    reg_rows = combined[combined['activity'] == 'registration']
    if not reg_rows.empty:
        reg_by_case = reg_rows.set_index(case_id_col)
        cols_to_fill = [c for c in combined.columns if c not in [case_id_col, 'activity', 'timestamp']]
        _filled_total = 0
        for col in cols_to_fill:
            if col in reg_by_case.columns:
                _before_missing = int(combined[col].isna().sum())
                combined[col] = combined[col].fillna(combined[case_id_col].map(reg_by_case[col]))
                _after_missing = int(combined[col].isna().sum())
                _filled_total += (_before_missing - _after_missing)
        if _filled_total > 0:
            print(f"Filled {_filled_total} missing attribute value(s) from registration event attributes.")

    # Update log_df
    log_df = combined
    print(f"Orders integrated. New event log size: {len(log_df)} events for {log_df[case_id_col].nunique()} cases.")
    display(log_df.head(10))

[orders] using prefiltered order events: 22381 row(s)
Removed 674 exact duplicate event row(s) (same case, activity, timestamp).
Filled 43414 missing attribute value(s) from registration event attributes.
Orders integrated. New event log size: 33812 events for 2421 cases.


,SEHRegistratienummer,activity,timestamp,Age,Gender,Specialismecode
0,0000155083,registration,2019-01-01 10:23:00,86.0,Vrouw,ORT
1,0000155083,arrivel,2019-01-01 10:45:00,86.0,Vrouw,ORT
2,0000155083,Medicatieverificatie SEH,2019-01-01 10:45:00,86.0,Vrouw,ORT
3,0000155083,start treatment,2019-01-01 10:45:00,86.0,Vrouw,ORT
4,0000155083,triage,2019-01-01 10:51:00,86.0,Vrouw,ORT
5,0000155083,Beleidsorder SEH,2019-01-01 11:37:00,86.0,Vrouw,ORT
6,0000155083,Radiologie,2019-01-01 11:37:00,86.0,Vrouw,ORT
7,0000155083,Radiologieaanvraag SEH,2019-01-01 11:37:00,86.0,Vrouw,ORT
8,0000155083,SEH Radiologie bespreking,2019-01-01 11:37:00,86.0,Vrouw,ORT
9,0000155083,Beleidsorder SEH,2019-01-01 12:39:00,86.0,Vrouw,ORT


### Variants (after orders integration)
- The event log has changed after integrating Orders.
- Below we compute per-case activity sequences from the current, combined `log_df` and show the top variants with percentages.
- This snapshot is taken right after Orders are merged, and before assigning resources.

In [15]:
# Variants after orders integration: compute per-case sequences and frequencies on the updated log
print("Variants (after orders integration)")

variant_counts, seq_per_case = compute_variant_counts(
    log_df,
    case_id_col=case_id_col,
    activity_col='activity',
    activity_tie_order={
        'registration': 0,
        'arrivel': 1,
        'triage': 2,
        # Orders effectively rank 3 via integration sort; tie-break handled earlier
        'start treatment': 4,
        'depart': 5,
    },
    compress=True,
)

total_cases = int(seq_per_case.shape[0])
variant_counts['percent_of_cases'] = (variant_counts['cases'] / max(1, total_cases) * 100).round(2)

print(f"Total cases: {total_cases}")
print(f"Total unique variants (after orders): {len(variant_counts)}")
TOP_SHOW = 20
print(f"Top {min(TOP_SHOW, len(variant_counts))} variants by cases (with coverage totals):")

# Prepare top-N with a totals row
_top_n = min(TOP_SHOW, len(variant_counts))
top_df = variant_counts.head(_top_n).copy()
coverage_cases = int(top_df['cases'].sum())
coverage_pct = round(coverage_cases / max(1, total_cases) * 100, 2)
totals_row = pd.DataFrame([
    {
        'variant': f'TOTAL (top {_top_n})',
        'cases': coverage_cases,
        'percent_of_cases': coverage_pct,
    }
])

display(pd.concat([top_df, totals_row], ignore_index=True))

Variants (after orders integration)
Total cases: 2421
Total unique variants (after orders): 1643
Top 20 variants by cases (with coverage totals):


,variant,cases,percent_of_cases
0,registration > arrivel > start treatment > triage > Pijnscore herhalen > depart > Pijnscore herhalen,55,2.27
1,registration > arrivel > start treatment > triage > Pijnscore herhalen > depart,36,1.49
2,registration > arrivel > Medicatieverificatie SEH > start treatment > triage > Pijnscore herhalen > depart > Pijnsco...,31,1.28
3,registration > arrivel > Medicatieverificatie SEH > start treatment > triage > Pijnscore herhalen > depart,29,1.20
4,registration > arrivel > start treatment > triage > Pijnscore herhalen > Afspraak maken (over 1 week) > Ontslagbelei...,24,0.99
5,registration > arrivel > start treatment > triage > Pijnscore herhalen > Beleidsorder SEH > Radiologie > Radiologiea...,20,0.83
6,registration > arrivel > start treatment > triage > depart,19,0.78
7,registration > arrivel > start treatment > triage > Pijnscore herhalen > depart > SEH Radiologie bespreking,19,0.78
8,registration > arrivel > Medicatieverificatie SEH > start treatment > triage > depart,16,0.66
9,registration > arrivel > start treatment > triage > Pijnscore herhalen > depart > Pijnscore herhalen > SEH Radiologi...,16,0.66


# Assign resources
Assign deterministic nurse/doctor resources per event using shift and weekend patterns; keep assignments stable via a seeded RNG.

### Define activity roles
- Below we list all activities present in the current `log_df` and assign a role to each: `nurse` or `doctor`.
- A default heuristic is used (e.g., "start treatment" → doctor; everything else → nurse). You can edit the mapping.
- The resulting `activity_role` dict will be used by the resource assignment step.

In [16]:
# List activities and assign a role per activity (doctor or nurse)

# Collect unique activities as they appear in the log
activities = (pd.Series(log_df['activity'])
                .dropna()
                .astype(str)
                .str.strip()
                .unique()
                .tolist())
activities = sorted(activities)

print(f"Found {len(activities)} activities:")
for a in activities:
    print(f" - {a}")

# Helper to normalize text (remove accents, lowercase, strip)

def _clean(text: str) -> str:
    return unicodedata.normalize("NFKD", str(text)).encode("ascii", "ignore").decode("ascii").lower().strip()

# Default heuristic: mark doctor-driven activities by keywords (EN/NL medical terms)
# - You can still override specific activities below with activity_role.update({...})

def _default_role_for(activity: str) -> str:
    s = _clean(activity)

    # Exact doctor-only names
    doctor_exact = {
        'start treatment',
    }

    # Substring keywords that strongly indicate a doctor action
    doctor_keywords = [
        # Clinical decision/assessment/consultations
        'consult', 'assessment', 'arts', 'dokter', 'physician', 'doctor',
        'beoordel',  # beoordeling (assessment)
        'diagnos',    # diagnose/diagnostic
        'anamnese',   # anamnesis
        'onderzoek',  # examination

        # Procedures/treatments/interventions
        'procedure', 'operat', 'ingreep', 'hecht', 'sutur', 'reduct',
        'intubat', 'reanim', 'resusc', 'immobil', 'gips', 'cast', 'splint',

        # Prescriptions/referrals/orders (typically initiated by doctor)
        'verwij', 'referr', 'prescrib', 'recept', 'medicat', 'order',

        # Imaging and diagnostics
        'x-ray', 'xray', 'rontg', 'radiolog', 'ct', 'mri', 'echo', 'ultra', 'ecg', 'ekg',

        # Laboratory
        'lab', 'bloed', 'blood', 'urine', 'kweek', 'culture', 'crp', 'troponin',
    ]

    if s in doctor_exact or any(kw in s for kw in doctor_keywords):
        return 'doctor'
    return 'nurse'

# Build mapping for all activities
activity_role = {a: _default_role_for(a) for a in activities}

# Example overrides (uncomment and adapt):
# activity_role.update({
#     'specific activity name': 'doctor',
#     'another activity': 'nurse',
# })

print("\nAssigned roles:")
for a in activities:
    print(f" - {a}: {activity_role[a]}")

print("\nSummary by role:")
print(dict(Counter(activity_role.values())))

Found 35 activities:
 - (P/M/O)EWS (1x dgs)
 - Aanvraag opname
 - Afspraak maken (over 1 week)
 - Anamnese in-/aanvullen ~
 - Bed- en deurkaartje bevestigd
 - Beleid klinisch
 - Beleidsorder SEH
 - Beleidsorder SEH-VPL
 - Bespreken patiëntveiligheidskaart
 - Bradenschaal
 - Consult op SEH
 - Controles bij opname
 - Defaecatielijst
 - Externe verslaglegging
 - Gewicht (1x wk zo)
 - Identificatiebandje aangeven
 - Inspectie insteekopening infuus
 - Labaanvraag SEH
 - Medicatieverificatie SEH
 - Ontslagbeleid Poliklinisch - SEH
 - Ontslagdatum gepland invullen
 - Opname
 - Pijnscore (3x dgs)
 - Pijnscore herhalen
 - Preoperatieve checklist + opname stopmoment
 - Preoperatieve screening
 - Radiologie
 - Radiologieaanvraag
 - Radiologieaanvraag SEH
 - SEH Radiologie bespreking
 - arrivel
 - depart
 - registration
 - start treatment
 - triage

Assigned roles:
 - (P/M/O)EWS (1x dgs): nurse
 - Aanvraag opname: nurse
 - Afspraak maken (over 1 week): nurse
 - Anamnese in-/aanvullen ~: doctor
 - 

### Activity counts

In [17]:
# Event counts by activity and by role
import pandas as pd

# Preconditions
if 'log_df' not in globals():
    raise RuntimeError("Event log 'log_df' not found. Please run the previous steps first.")
if 'activity_role' not in globals():
    raise RuntimeError("'activity_role' not defined. Please run the 'Define activity roles' cell before this one.")

# Total events
total_events = int(len(log_df))
print(f"Total events: {total_events}")

# Counts by activity
events_by_activity = (log_df['activity']
                      .value_counts()
                      .rename_axis('activity')
                      .reset_index(name='events'))
if total_events:
    events_by_activity['percent_of_events'] = (events_by_activity['events'] / total_events * 100).round(2)

# Role series from mapping
role_s = log_df['activity'].map(lambda a: activity_role.get(a, 'nurse')).rename('role')

# Counts by role
events_by_role = (role_s.value_counts()
                  .rename_axis('role')
                  .reset_index(name='events'))
if total_events:
    events_by_role['percent_of_events'] = (events_by_role['events'] / total_events * 100).round(2)

print("\nEvents by activity (top 25 shown):")
display(events_by_activity.head(25))

print("\nEvents by role:")
display(events_by_role)


Total events: 33812

Events by activity (top 25 shown):


,activity,events,percent_of_events
0,Pijnscore herhalen,3607,10.67
1,registration,2421,7.16
2,start treatment,2421,7.16
3,triage,2421,7.16
4,depart,2421,7.16
5,arrivel,2421,7.16
6,Beleidsorder SEH,2295,6.79
7,Radiologie,2012,5.95
8,Radiologieaanvraag SEH,1972,5.83
9,Externe verslaglegging,1744,5.16



Events by role:


,role,events,percent_of_events
0,nurse,19920,58.91
1,doctor,13892,41.09


# Staffing rationale and seasonal patterns

We size the number of nurse and doctor resources from observed demand and simple, transparent assumptions:

- Compute the time coverage of the log (from min to max timestamp) and average events/day by role.
- Set target throughput per staff per shift:
  - Nurses: 15 events/shift (triage, registration, orders support, etc.).
  - Doctors: 8 events/shift (assessments, procedures, orders initiation).
- Estimate baseline staff-per-shift = ceil(events_per_day / target_per_shift).
- Account for seasonal peaks: winter +20% demand, summer −15%, spring/autumn baseline.
- Recommend roster size as the peak staff-per-shift × number of shifts (3), with a small buffer.

Assignment patterns in the next cell reflect seasonality and shifts:
- Three shifts: day [07–15), evening [15–23), night [23–07).
- Weekends bias toward higher doctor coverage at night/evening.
- Seasonal active-roster fractions (e.g., a larger fraction of the roster active in winter), rotating weekly so names vary naturally.

You can override the recommended counts by setting `n_nurses` / `n_doctors` before the generation cell.

In [18]:
# Compute recommended staffing from demand and set defaults
import math
import pandas as pd

# Preconditions
if 'log_df' not in globals():
    raise RuntimeError("Event log 'log_df' not found. Please run previous steps.")
if 'activity_role' not in globals():
    raise RuntimeError("'activity_role' not defined. Please run the 'Define activity roles' cell.")

# Role series and counts
role_s = pd.Series(log_df['activity']).map(lambda a: activity_role.get(a, 'nurse')).rename('role')
role_counts = role_s.value_counts()
nurse_events = int(role_counts.get('nurse', 0))
doctor_events = int(role_counts.get('doctor', 0))

# Coverage window
_ts = pd.to_datetime(log_df['timestamp'], errors='coerce')
min_ts, max_ts = _ts.min(), _ts.max()
if pd.isna(min_ts) or pd.isna(max_ts):
    raise RuntimeError("Timestamps missing; cannot compute coverage window.")
days = max(1, int((max_ts.normalize() - min_ts.normalize()).days) + 1)

# Events/day by role
nurse_epd = nurse_events / days
doctor_epd = doctor_events / days

# Throughput targets (events per staff per shift)
NURSE_TARGET_PER_SHIFT = 15
DOCTOR_TARGET_PER_SHIFT = 8

# Baseline staff-per-shift
nurse_staff_shift = math.ceil(nurse_epd / max(1, NURSE_TARGET_PER_SHIFT))
doctor_staff_shift = math.ceil(doctor_epd / max(1, DOCTOR_TARGET_PER_SHIFT))

# Peak multipliers to cover winter; we size to peak
NURSE_PEAK_FACTOR = 1.20
DOCTOR_PEAK_FACTOR = 1.15

nurse_staff_shift_peak = max(1, math.ceil(nurse_staff_shift * NURSE_PEAK_FACTOR))
doctor_staff_shift_peak = max(1, math.ceil(doctor_staff_shift * DOCTOR_PEAK_FACTOR))

# Recommend roster sizes (3 shifts for nurses; 2 for doctors typical)
recommended_n_nurses = max(6, nurse_staff_shift_peak * 3)
recommended_n_doctors = max(3, doctor_staff_shift_peak * 2)

print("Demand-driven staffing suggestion:")
print(f"- Coverage: {days} day(s) from {min_ts.date()} to {max_ts.date()}")
print(f"- Nurse events: {nurse_events:,}  (~{nurse_epd:.1f}/day)")
print(f"- Doctor events: {doctor_events:,} (~{doctor_epd:.1f}/day)")
print(f"- Baseline staff/shift → nurses={nurse_staff_shift}, doctors={doctor_staff_shift}")
print(f"- Peak-adjusted staff/shift → nurses={nurse_staff_shift_peak}, doctors={doctor_staff_shift_peak}")
print(f"- Recommended unique names → nurses={recommended_n_nurses}, doctors={recommended_n_doctors}")

print("The generation step will use the recommended values by default; define n_nurses/n_doctors only if you explicitly want to override.")

Demand-driven staffing suggestion:
- Coverage: 639 day(s) from 2019-01-01 to 2020-09-30
- Nurse events: 19,920  (~31.2/day)
- Doctor events: 13,892 (~21.7/day)
- Baseline staff/shift → nurses=3, doctors=3
- Peak-adjusted staff/shift → nurses=4, doctors=4
- Recommended unique names → nurses=12, doctors=8
The generation step will use the recommended values by default; define n_nurses/n_doctors only if you explicitly want to override.


### Generate Resources

In [19]:
n_nurses = int(globals().get('recommended_n_nurses', globals().get('n_nurses', 6)))
n_doctors = int(globals().get('recommended_n_doctors', globals().get('n_doctors', 3)))

# -----------------------------
# Kept knobs (no season needed)
# -----------------------------
RHO_TARGET = 0.80  # target utilization
SERVICE_RATE_PER_ROLE = {  # completions/hour per resource
    "nurse": 6.0,
    "doctor": 3.0,
}
TAU_BY_SHIFT = {"day": 1.2, "evening": 1.0, "night": 0.7, "unknown": 1.0}
CONTINUITY_GAMMA = 2.0
SHIFT_STEP = {"day": 3, "evening": 5, "night": 7, "unknown": 3}  # for full-roster rotation

# Helper: generate alphabetic suffixes: A..Z, AA..ZZ, AAA..
from typing import List
def _alpha_suffixes(n: int) -> List[str]:
    def _to_letters(idx: int) -> str:
        s, x = "", idx + 1
        while x > 0:
            x, rem = divmod(x - 1, 26)
            s = chr(ord('A') + rem) + s
        return s
    return [_to_letters(i) for i in range(max(0, int(n)))]

# Names
nurse_names = [f"nurse_{s}" for s in _alpha_suffixes(n_nurses)]
doctor_names = [f"doctor_{s}" for s in _alpha_suffixes(n_doctors)]

In [20]:
# Prefer recommended counts; only fall back if not available
n_nurses = int(globals().get('recommended_n_nurses', globals().get('n_nurses', 6)))
n_doctors = int(globals().get('recommended_n_doctors', globals().get('n_doctors', 3)))

# -----------------------------
# Kept knobs (no season needed)
# -----------------------------
RHO_TARGET = 0.80  # target utilization
SERVICE_RATE_PER_ROLE = {  # completions/hour per resource
    "nurse": 6.0,
    "doctor": 3.0,
}
TAU_BY_SHIFT = {"day": 1.2, "evening": 1.0, "night": 0.7, "unknown": 1.0}
CONTINUITY_GAMMA = 2.0
SHIFT_STEP = {"day": 3, "evening": 5, "night": 7, "unknown": 3}  # for full-roster rotation

# Helper: generate alphabetic suffixes: A..Z, AA..ZZ, AAA..
from typing import List
def _alpha_suffixes(n: int) -> List[str]:
    def _to_letters(idx: int) -> str:
        s, x = "", idx + 1
        while x > 0:
            x, rem = divmod(x - 1, 26)
            s = chr(ord('A') + rem) + s
        return s
    return [_to_letters(i) for i in range(max(0, int(n)))]

# Names
nurse_names = [f"nurse_{s}" for s in _alpha_suffixes(n_nurses)]
doctor_names = [f"doctor_{s}" for s in _alpha_suffixes(n_doctors)]

# Deterministic RNG per event
def rng_for_row(row, extra_key: str = ""):
    date_str = row["timestamp"].strftime("%Y-%m-%d") if pd.notna(row["timestamp"]) else "NA"
    key = f"{row[case_id_col]}|{row['activity']}|{date_str}|{extra_key}"
    seed = int(hashlib.sha1(key.encode("utf-8")).hexdigest(), 16) % (2**32)
    return np.random.default_rng(seed)

# Shift + weekend + week parity (parity kept only for pivot rotation)
def shift_and_week(row):
    ts = row["timestamp"]
    if pd.isna(ts):
        return "unknown", False, 0
    hour = ts.hour
    if 7 <= hour < 15:
        shift = "day"
    elif 15 <= hour < 23:
        shift = "evening"
    else:
        shift = "night"
    is_weekend = ts.dayofweek >= 5
    week_parity = int(ts.isocalendar().week) % 2
    return shift, is_weekend, week_parity

# Activity -> role mapping
if 'activity_role' not in globals():
    activity_role = {
        "registration": "nurse",
        "arrivel": "nurse",
        "triage": "nurse",
        "start treatment": "doctor",
        "depart": "nurse",
    }

# -----------------------------
# Demand from the actual log
# -----------------------------
log_df = log_df.sort_values("timestamp")  # IMPORTANT for continuity
_tmp = log_df.copy()
_tmp["shift"] = _tmp.apply(lambda r: shift_and_week(r)[0], axis=1)
_tmp["role"]  = _tmp["activity"].map(lambda a: activity_role.get(a, "nurse"))
_tmp["date"]  = _tmp["timestamp"].dt.date

def _shift_len_hours(shift: str) -> float:
    return 8.0  # day/evening/night = 8h

demand_counts = (
    _tmp.groupby(["date", "shift", "role"])
        .size()
        .rename("tasks")
        .reset_index()
)
demand_counts["lambda_per_hour"] = demand_counts["tasks"] / demand_counts["shift"].map(_shift_len_hours)
DEMAND_LOOKUP = {
    (row["date"], row["shift"], row["role"]): float(row["lambda_per_hour"])
    for _, row in demand_counts.iterrows()
}

def demand_lambda(ts: pd.Timestamp, shift: str, role: str) -> float:
    """Return measured tasks/hour; if missing, assume 0."""
    if pd.isna(ts):
        return 0.0
    return DEMAND_LOOKUP.get((ts.date(), shift, role), 0.0)

def target_k(n_total: int, role: str, ts: pd.Timestamp, shift: str) -> int:
    """k = ceil(lambda / (rho*mu)), clipped to [1, n_total]."""
    if n_total <= 0:
        return 0
    lam = demand_lambda(ts, shift, role)                  # tasks/hour
    mu  = SERVICE_RATE_PER_ROLE.get(role, 4.0)            # tasks/hour per resource
    k = int(np.ceil(lam / max(1e-9, (RHO_TARGET * mu))))
    return int(np.clip(max(1, k), 1, n_total))            # at least 1 on duty

# -----------------------------
# Full-roster rotation + scatter
# -----------------------------
def active_indices(n: int, k: int, ts: pd.Timestamp, shift: str) -> list[int]:
    if n <= 0 or k <= 0:
        return []
    if k >= n:
        return list(range(n))

    week_num = int(ts.isocalendar().week) if pd.notna(ts) else 0
    step = SHIFT_STEP.get(shift, 3)
    start = (step * week_num) % n
    gap = n / k

    # Use floor, not round, to stay within [0, n-1]
    vals = (start + np.arange(k) * gap) % n
    idxs = np.floor(vals).astype(int).tolist()

    # Deduplicate (rare when n/k is not integer), then fill forward
    seen = set()
    unique = []
    for v in idxs:
        if v not in seen:
            unique.append(v)
            seen.add(v)

    cur = start
    while len(unique) < k:
        cur = (cur + 1) % n
        if cur not in seen:
            unique.append(int(cur))
            seen.add(int(cur))

    # Final safety: clamp (should be unnecessary now, but cheap)
    return [i % n for i in unique]

# -----------------------------
# Softmax weights around pivot
# -----------------------------
def _softmax_weights(m: int, pivot: int, shift: str) -> np.ndarray:
    if m <= 0:
        return np.array([])
    tau = TAU_BY_SHIFT.get(shift, 1.0)
    idx = np.arange(m)
    d = np.minimum(np.abs(idx - pivot), m - np.abs(idx - pivot))  # circular distance
    w = np.exp(-(d ** 2) / (2 * tau * tau))
    return w / w.sum()

def nurse_weights(n: int, shift: str, is_weekend: bool, week_parity: int) -> np.ndarray:
    if n <= 0:
        return np.array([])
    if shift == "night":
        pivot = (2 + week_parity) % n if n >= 3 else (week_parity % n)
    elif is_weekend:
        pivot = (1 + week_parity) % n if n >= 2 else 0
    else:
        pivot = (0 + week_parity) % n if n >= 2 else 0
    return _softmax_weights(n, pivot, shift)

def doctor_weights(n: int, shift: str, is_weekend: bool, week_parity: int) -> np.ndarray:
    if n <= 0:
        return np.array([])
    pivot = 0
    if n >= 2:
        pivot = (1 if (is_weekend or shift == "night") else 0)
        pivot = (pivot + week_parity) % n
    return _softmax_weights(n, pivot, shift)

# Weighted choice
def pick(pool, weights, rng):
    idx = rng.choice(len(pool), p=weights)
    return pool[int(idx)]

# -----------------------------
# Continuity memory
# -----------------------------
_last_resource_for = {}  # (case_id, role, shift) -> resource_name

# Assignment
def assign_resource(row):
    role = activity_role.get(row["activity"], "nurse")
    shift, is_weekend, week_parity = shift_and_week(row)
    ts = row["timestamp"]

    # Determine pool size from measured demand
    if role == "nurse":
        n_total = len(nurse_names)
        k = target_k(n_total, role, ts, shift)
        idxs = active_indices(n_total, k, ts, shift)
        pool = [nurse_names[i] for i in idxs]
        w = nurse_weights(len(pool), shift, is_weekend, week_parity)
    else:
        n_total = len(doctor_names)
        k = target_k(n_total, role, ts, shift)
        idxs = active_indices(n_total, k, ts, shift)
        pool = [doctor_names[i] for i in idxs]
        w = doctor_weights(len(pool), shift, is_weekend, week_parity)

    # Continuity boost within (case, role, shift)
    case_key = (row[case_id_col], role, shift)
    prev = _last_resource_for.get(case_key)
    if prev in pool:
        j = pool.index(prev)
        w[j] *= CONTINUITY_GAMMA
        w /= w.sum()

    rng = rng_for_row(row, extra_key=f"{shift}|{is_weekend}|{week_parity}")  # season removed
    chosen = pick(pool, w, rng)

    _last_resource_for[case_key] = chosen
    return chosen

# Apply to event log
log_df["resource"] = log_df.apply(assign_resource, axis=1)

# Summary
print("Resource distribution by activity (top 10 shown):")
summary = (log_df.groupby(["activity", "resource"]).size().sort_values(ascending=False))
print(summary.head(10))
display(log_df.head(10))


Resource distribution by activity (top 10 shown):
activity            resource
Pijnscore herhalen  nurse_D     530
                    nurse_J     525
                    nurse_G     500
                    nurse_A     500
registration        nurse_D     415
arrivel             nurse_D     407
registration        nurse_J     406
triage              nurse_D     398
                    nurse_G     385
registration        nurse_G     375
dtype: int64


,SEHRegistratienummer,activity,timestamp,Age,Gender,Specialismecode,resource
0,0000155083,registration,2019-01-01 10:23:00,86.0,Vrouw,ORT,nurse_D
1,0000155083,arrivel,2019-01-01 10:45:00,86.0,Vrouw,ORT,nurse_D
2,0000155083,Medicatieverificatie SEH,2019-01-01 10:45:00,86.0,Vrouw,ORT,doctor_D
3,0000155083,start treatment,2019-01-01 10:45:00,86.0,Vrouw,ORT,doctor_D
16,0000155084,registration,2019-01-01 10:50:00,92.0,Vrouw,ORT,nurse_D
4,0000155083,triage,2019-01-01 10:51:00,86.0,Vrouw,ORT,nurse_D
5,0000155083,Beleidsorder SEH,2019-01-01 11:37:00,86.0,Vrouw,ORT,doctor_D
6,0000155083,Radiologie,2019-01-01 11:37:00,86.0,Vrouw,ORT,doctor_F
7,0000155083,Radiologieaanvraag SEH,2019-01-01 11:37:00,86.0,Vrouw,ORT,doctor_A
8,0000155083,SEH Radiologie bespreking,2019-01-01 11:37:00,86.0,Vrouw,ORT,doctor_D


In [190]:

# # Prefer recommended counts; only fall back if not available
# n_nurses = int(globals().get('recommended_n_nurses', globals().get('n_nurses', 6)))
# n_doctors = int(globals().get('recommended_n_doctors', globals().get('n_doctors', 3)))

# # -----------------------------
# # Kept knobs (no season needed)
# # -----------------------------
# RHO_TARGET = 0.80  # target utilization
# SERVICE_RATE_PER_ROLE = {  # completions/hour per resource
#     "nurse": 6.0,
#     "doctor": 3.0,
# }
# TAU_BY_SHIFT = {"day": 1.2, "evening": 1.0, "night": 0.7, "unknown": 1.0}
# CONTINUITY_GAMMA = 2.0
# SHIFT_STEP = {"day": 3, "evening": 5, "night": 7, "unknown": 3}  # for full-roster rotation

# # Helper: generate alphabetic suffixes: A..Z, AA..ZZ, AAA..
# from typing import List

# def _alpha_suffixes(n: int) -> List[str]:
#     def _to_letters(idx: int) -> str:
#         # Excel-like column naming: 0->A, 25->Z, 26->AA, etc.
#         s = ""
#         x = idx + 1
#         while x > 0:
#             x, rem = divmod(x - 1, 26)
#             s = chr(ord('A') + rem) + s
#         return s
#     return [_to_letters(i) for i in range(max(0, int(n)))]

# # Ensure names use only letters after the underscore, lowercase role prefix
# nurse_names = [f"nurse_{s}" for s in _alpha_suffixes(n_nurses)]
# doctor_names = [f"doctor_{s}" for s in _alpha_suffixes(n_doctors)]

# # Helper: deterministic RNG per event

# def rng_for_row(row, extra_key: str = ""):
#     # Seed based on case, activity, date, and shift key so assignment is stable across runs
#     date_str = row["timestamp"].strftime("%Y-%m-%d") if pd.notna(row["timestamp"]) else "NA"
#     key = f"{row[case_id_col]}|{row['activity']}|{date_str}|{extra_key}"
#     seed = int(hashlib.sha1(key.encode("utf-8")).hexdigest(), 16) % (2**32)
#     return np.random.default_rng(seed)

# # Shift + weekend + week parity (parity kept only for pivot rotation)
# def shift_and_week(row):
#     ts = row["timestamp"]
#     if pd.isna(ts):
#         return "unknown", False, 0
#     hour = ts.hour
#     if 7 <= hour < 15:
#         shift = "day"
#     elif 15 <= hour < 23:
#         shift = "evening"
#     else:
#         shift = "night"
#     is_weekend = ts.dayofweek >= 5
#     week_parity = int(ts.isocalendar().week) % 2
#     return shift, is_weekend, week_parity

# # Activity -> role mapping
# if 'activity_role' not in globals():
#     activity_role = {
#         "registration": "nurse",
#         "arrivel": "nurse",
#         "triage": "nurse",
#         "start treatment": "doctor",
#         "depart": "nurse",
#     }


# def active_indices(n: int, fraction: float, parity: int) -> list[int]:
#     if n <= 0:
#         return []
#     k = max(1, int(round(n * float(fraction))))
#     start = (2 * parity) % n  # rotate weekly
#     return [int((start + i) % n) for i in range(k)]

# # Dynamic weight builders so patterns scale with pool size

# def nurse_weights(n: int, shift: str, is_weekend: bool, week_parity: int) -> np.ndarray:
#     if n <= 0:
#         return np.array([])
#     # Choose a pivot index for the most-likely nurse depending on shift/weekend and parity
#     if shift == "night":
#         pivot = (2 + week_parity) % n if n >= 3 else (week_parity % n)
#     elif is_weekend:
#         pivot = (1 + week_parity) % n if n >= 2 else 0
#     else:
#         pivot = (0 + week_parity) % n if n >= 2 else 0
#     w = np.full(n, 0.1)
#     w[pivot] += 0.4  # primary
#     if n >= 2:
#         w[(pivot+1) % n] += 0.2  # secondary
#     if n >= 3:
#         w[(pivot+2) % n] += 0.1  # tertiary
#     w = w / w.sum()
#     return w


# def doctor_weights(n: int, shift: str, is_weekend: bool, week_parity: int) -> np.ndarray:
#     if n <= 0:
#         return np.array([])
#     # Favor second doctor on nights/weekends, rotate weekly if possible
#     if n >= 2:
#         pivot = 1 if (is_weekend or shift == "night") else 0
#         pivot = (pivot + week_parity) % n
#     else:
#         pivot = 0
#     w = np.full(n, 0.2)
#     w[pivot] += 0.4  # primary
#     if n >= 2:
#         w[(pivot+1) % n] += 0.2  # secondary
#     w = w / w.sum()
#     return w

# # Weighted choice wrapper

# def pick(pool, weights, rng):
#     idx = rng.choice(len(pool), p=weights)
#     return pool[int(idx)]

# # Activity -> role mapping (respect pre-defined mapping if present)
# if 'activity_role' not in globals():
#     activity_role = {
#         "registration": "nurse",
#         "arrivel": "nurse",
#         "triage": "nurse",
#         "start treatment": "doctor",
#         "depart": "nurse",
#     }

# # Assignment logic per row

# def assign_resource(row):
#     role = activity_role.get(row["activity"], "nurse")
#     shift, is_weekend, week_parity = shift_and_week(row)
#     ts = row["timestamp"]
#     season = season_of(ts)

#     # Determine active roster based on season/shift/weekend
#     if role == "nurse":
#         frac = nurse_active_fraction.get(season, nurse_active_fraction["spring"]).get(shift, 0.9)
#         if is_weekend:
#             frac = max(frac, 0.95)
#         indices = active_indices(len(nurse_names), frac, week_parity)
#         pool = [nurse_names[i] for i in indices]
#         w = nurse_weights(len(pool), shift, is_weekend, week_parity)
#     else:
#         frac = doctor_active_fraction.get(season, doctor_active_fraction["spring"]).get(shift, 0.9)
#         if is_weekend and shift in ("evening", "night"):
#             frac = max(frac, 1.0)
#         indices = active_indices(len(doctor_names), frac, week_parity)
#         pool = [doctor_names[i] for i in indices]
#         w = doctor_weights(len(pool), shift, is_weekend, week_parity)

#     rng = rng_for_row(row, extra_key=f"{shift}|{is_weekend}|{week_parity}|{season}")
#     return pick(pool, w, rng)

# # Apply to event log
# log_df["resource"] = log_df.apply(assign_resource, axis=1)

# # Concise distribution summary
# print("Resource distribution by activity (top 10 shown):")
# summary = (log_df.groupby(["activity", "resource"])  # type: ignore
#                     .size()
#                     .sort_values(ascending=False))
# print(summary.head(10))

# display(log_df.head(10))

In [191]:
# from math import exp
# import numpy as np
# import pandas as pd

# # Configuration knobs (feel free to move these near other global config cells)
# TIME_PREF_CENTERS_FRAC = [0.10, 0.45, 0.80]   # fractional positions of time range for resource preference peaks
# TIME_PREF_MIN_SPREAD_MINUTES = 5.0            # lower bound on Gaussian sigma (minutes)
# MAX_RESOURCES_PER_ROLE = 50                   # safety cap if role list is huge
# CONTINUITY_GAMMA_TIME_DEP = 2  # reuse existing continuity strength
# RNG_SEED_TIME_DEP = 123

# rng_time_dep = np.random.default_rng(RNG_SEED_TIME_DEP)

# # Precompute relative time per event if not already present
# _rel_col = 'relative_timestamp_from_start'
# if _rel_col not in log_df.columns:
#     # Use first timestamp per case
#     _first_ts = log_df.groupby(case_id_col)['timestamp'].transform('min')
#     log_df[_rel_col] = (log_df['timestamp'] - _first_ts).dt.total_seconds() / 60.0
# else:
#     log_df[_rel_col] = pd.to_numeric(log_df[_rel_col], errors='coerce').fillna(0.0)

# # Establish max (e.g. 95th percentile) to define range for centers
# _time_range_95 = float(log_df[_rel_col].quantile(0.95)) if len(log_df) else 60.0
# _time_range_95 = max(_time_range_95, 60.0)  # at least 1 hour to avoid degenerate ranges

# _centers_minutes = [c * _time_range_95 for c in TIME_PREF_CENTERS_FRAC]
# _sigma_minutes = max(TIME_PREF_MIN_SPREAD_MINUTES, 0.25 * (_centers_minutes[-1] - _centers_minutes[0]))

# # Build per-role resource lists from existing variables
# nurse_list  = nurse_names[:MAX_RESOURCES_PER_ROLE]
# doctor_list = doctor_names[:MAX_RESOURCES_PER_ROLE]

# # Assign each resource a center (cycle through centers if more resources than centers)
# _resource_centers = {}
# for role_list in [nurse_list, doctor_list]:
#     for i, r in enumerate(role_list):
#         _resource_centers[r] = _centers_minutes[i % len(_centers_minutes)]

# # Helper: compute unnormalized time weight for a resource at elapsed minutes t
# _DEF_BASE_EPS = 1e-6

# def _time_weight(resource: str, t_minutes: float) -> float:
#     c = _resource_centers.get(resource)
#     if c is None:
#         # fallback flat weight for resources not in center mapping
#         return 1.0
#     diff = t_minutes - c
#     return exp(-0.5 * (diff / (_sigma_minutes + 1e-9)) ** 2) + _DEF_BASE_EPS

# # Maintain per-case previous resource for continuity
# _prev_resource_by_case = {}

# # Optionally cache per-activity role (already in activity_role)
# # activity_role: dict mapping Activity -> role string (e.g. 'nurse' or 'doctor')

# # Function: drop-in replacement for previous assign_resource

# def assign_resource_time_dependent(row: pd.Series) -> str:
#     """Time-dependent resource assignment creating R–T dependence within activities.
#     Uses a Gaussian time preference per resource and a continuity boost.

#     Assumptions:
#       - `row` contains at least: case_id_col, 'Activity', 'time:timestamp'.
#       - Global lists nurse_names, doctor_names define candidate resources per role.
#       - activity_role maps activity name to either 'nurse' or 'doctor'; default fallback to nurse list if unknown.
#     """
#     case_id = row[case_id_col]
#     activity = row['Activity'] if 'Activity' in row else row.get('concept:name')
#     t_minutes = float(row[_rel_col]) if _rel_col in row else 0.0

#     role = activity_role.get(activity, 'nurse').lower()
#     if role.startswith('nurse'):
#         candidates = nurse_list
#     elif role.startswith('doctor'):
#         candidates = doctor_list
#     else:
#         # fallback merges both pools to avoid empty candidate set
#         candidates = nurse_list + doctor_list
#         if not candidates:
#             return 'UNKNOWN'

#     if not candidates:
#         return 'NO_RESOURCE'

#     # Build time-conditioned weights
#     weights = np.array([_time_weight(r, t_minutes) for r in candidates], dtype=float)

#     # Continuity boost if previous resource same case
#     prev_r = _prev_resource_by_case.get(case_id)
#     if prev_r in candidates and CONTINUITY_GAMMA_TIME_DEP > 1.0:
#         j = candidates.index(prev_r)
#         weights[j] *= CONTINUITY_GAMMA_TIME_DEP

#     # Normalize and sample
#     total = weights.sum()
#     if total <= 0 or not np.isfinite(total):
#         weights = np.ones(len(candidates), dtype=float)
#         total = weights.sum()
#     probs = weights / total
#     chosen = rng_time_dep.choice(candidates, p=probs)

#     # Update continuity state
#     _prev_resource_by_case[case_id] = chosen
#     return chosen

# # Apply the new function to assign resources
# log_df['Resource'] = log_df.apply(assign_resource_time_dependent, axis=1)
# print("[assign_resource_time_dependent] Assigned resources with time-conditioned + continuity logic.")
# display(log_df.head(10))

### Active roster preview
Use this cell to preview who is on-duty for a given timestamp, per role (nurse/doctor).
- It shows: season, shift, weekend flag, week parity, the active fraction, the rotating start index, and the active names.
- It also shows assignment weights within the active pool (higher weight = more likely to be assigned).
- The result is deterministic for the same timestamp and data (stable seeds).

In [192]:
# # Preview parameters
# PREVIEW_TS = None  # Set to e.g., '2012-01-10 10:30'; if None, uses the first timestamp in the log
# MAX_SHOW = 10      # How many top-weighted names to show per role

# import pandas as pd
# import numpy as np

# def _pick_preview_timestamp():
#     if PREVIEW_TS is not None:
#         return pd.to_datetime(PREVIEW_TS)
#     if 'log_df' in globals() and 'timestamp' in log_df.columns:
#         ts_series = pd.to_datetime(log_df['timestamp'], errors='coerce').dropna().sort_values()
#         if not ts_series.empty:
#             return ts_series.iloc[0]
#     # Fallback: now (note: season/shift logic still works)
#     return pd.Timestamp.now()

# def _active_roster_for_role(role, ts, shift, is_weekend, week_parity, season):
#     if role not in ('nurse', 'doctor'):
#         raise ValueError('role must be nurse or doctor')
#     names = nurse_names if role == 'nurse' else doctor_names
#     n = len(names)
#     if n == 0:
#         return {
#             'fraction': 0.0, 'n': 0, 'k': 0, 'start': 0, 'indices': [], 'pool': [], 'weights': np.array([])
#         }

#     if role == 'nurse':
#         frac = nurse_active_fraction.get(season, nurse_active_fraction['spring']).get(shift, 0.9)
#         if is_weekend:
#             frac = max(frac, 0.95)
#         k = max(1, int(round(n * float(frac))))
#         start = (2 * int(week_parity)) % n
#         indices = [int((start + i) % n) for i in range(k)]
#         pool = [names[i] for i in indices]
#         w = nurse_weights(len(pool), shift, is_weekend, week_parity) if len(pool) > 0 else np.array([])
#     else:  # doctor
#         frac = doctor_active_fraction.get(season, doctor_active_fraction['spring']).get(shift, 0.9)
#         if is_weekend and shift in ('evening', 'night'):
#             frac = max(frac, 1.0)
#         k = max(1, int(round(n * float(frac))))
#         start = (2 * int(week_parity)) % n
#         indices = [int((start + i) % n) for i in range(k)]
#         pool = [names[i] for i in indices]
#         w = doctor_weights(len(pool), shift, is_weekend, week_parity) if len(pool) > 0 else np.array([])

#     return {'fraction': float(frac), 'n': n, 'k': k, 'start': start, 'indices': indices, 'pool': pool, 'weights': w}

# # Determine context for preview
# ts = _pick_preview_timestamp()
# row = pd.Series({'timestamp': ts})
# shift, is_weekend, week_parity = shift_and_week(row)
# season = season_of(ts)

# print(f"Previewing active roster at {ts} → shift={shift}, weekend={is_weekend}, parity={week_parity}, season={season}")

# for role in ('nurse', 'doctor'):
#     info = _active_roster_for_role(role, ts, shift, is_weekend, week_parity, season)
#     print(f"\nRole: {role}")
#     print(f"- total names (n): {info['n']}")
#     print(f"- active fraction (f): {info['fraction']:.2f}")
#     print(f"- active count (k): {info['k']}")
#     print(f"- rotating start index: {info['start']}")
#     print(f"- active names: {len(info['pool'])}")
#     if info['pool'] and len(info['weights']) == len(info['pool']):
#         dfw = pd.DataFrame({'name': info['pool'], 'weight': info['weights']})
#         dfw['weight_pct'] = (dfw['weight'] / dfw['weight'].sum() * 100).round(2) if dfw['weight'].sum() else 0
#         display(dfw.sort_values('weight', ascending=False).head(int(MAX_SHOW)))
#     else:
#         print('(no active names to show)')

### Add relative timestamp between activities
Compute the per-event delay since the previous event in the same case using the utility function, and attach it as `relative_timestamp_from_previous_activity`.

In [21]:
# Ensure canonical timestamp column name
# Prefer 'time:timestamp' across the pipeline; rename if an older 'timestamp' exists
if 'time:timestamp' not in log_df.columns and 'timestamp' in log_df.columns:
    log_df = log_df.rename(columns={'timestamp': 'time:timestamp'})
    log_df = log_df.rename(columns={'activity': 'Activity'})
    log_df = log_df.rename(columns={'resource': 'Resource'})

# Parse and sort to satisfy downstream utilities
log_df['time:timestamp'] = pd.to_datetime(log_df['time:timestamp'], errors='coerce')

log_df = log_df.sort_values([case_id_col, 'time:timestamp']).reset_index(drop=True)

In [22]:
# Add relative timestamp between activities and from log start using the project utilities
from src.preprocess_log import add_relative_timestamp_between_activities, add_trace_attr_relative_timestamp_to_first_activity

# At this point, 'time:timestamp' should be the canonical timestamp column (see prior cell)
# Ensure correct dtype and sort order
log_df['time:timestamp'] = pd.to_datetime(log_df['time:timestamp'], errors='coerce')
log_df = log_df.sort_values([case_id_col, 'time:timestamp']).reset_index(drop=True)

# 1) Minutes since previous event in the same case
log_df = add_relative_timestamp_between_activities(
    log_df,
    trace_key=case_id_col,
    timestamp_key='time:timestamp',
    custom_timestamp_key='relative_timestamp_from_previous_activity',
)

log_df = add_trace_attr_relative_timestamp_to_first_activity(
    log_df,
    trace_key=case_id_col,
    timestamp_key='time:timestamp',
    custom_timestamp_key='relative_timestamp_from_start',
)

log_df["label"] = log_df['Gender']

print("Added column: 'relative_timestamp_from_previous_activity' (minutes)")

display(log_df.head(10))

Added column: 'relative_timestamp_from_previous_activity' (minutes)


,SEHRegistratienummer,Activity,time:timestamp,Age,Gender,Specialismecode,Resource,relative_timestamp_from_previous_activity,relative_timestamp_from_start,label
0,0000155083,registration,2019-01-01 10:23:00,86.0,Vrouw,ORT,nurse_D,0.0,0.0,Vrouw
1,0000155083,arrivel,2019-01-01 10:45:00,86.0,Vrouw,ORT,nurse_D,22.0,0.0,Vrouw
2,0000155083,Medicatieverificatie SEH,2019-01-01 10:45:00,86.0,Vrouw,ORT,doctor_D,0.0,0.0,Vrouw
3,0000155083,start treatment,2019-01-01 10:45:00,86.0,Vrouw,ORT,doctor_D,0.0,0.0,Vrouw
4,0000155083,triage,2019-01-01 10:51:00,86.0,Vrouw,ORT,nurse_D,6.0,0.0,Vrouw
5,0000155083,Beleidsorder SEH,2019-01-01 11:37:00,86.0,Vrouw,ORT,doctor_D,46.0,0.0,Vrouw
6,0000155083,Radiologie,2019-01-01 11:37:00,86.0,Vrouw,ORT,doctor_F,0.0,0.0,Vrouw
7,0000155083,Radiologieaanvraag SEH,2019-01-01 11:37:00,86.0,Vrouw,ORT,doctor_A,0.0,0.0,Vrouw
8,0000155083,SEH Radiologie bespreking,2019-01-01 11:37:00,86.0,Vrouw,ORT,doctor_D,0.0,0.0,Vrouw
9,0000155083,Radiologieaanvraag SEH,2019-01-01 12:39:00,86.0,Vrouw,ORT,doctor_A,62.0,0.0,Vrouw


In [23]:
# Compute per-event relative time from case start (minutes)
import pandas as pd
from typing import Optional

# Expect these to already exist from earlier cells:
# - df: events DataFrame
#! - case_id_col: name of the case id column (e.g., 'case:concept:name')
#! - ts_col: name of the timestamp column (e.g., 'time:timestamp')

assert 'df' in globals(), "DataFrame 'df' not found in notebook scope"
assert 'case_id_col' in globals(), "Variable 'case_id_col' not found"

# Ensure timestamp is datetime
if not pd.api.types.is_datetime64_any_dtype(log_df["time:timestamp"]):
    log_df["time:timestamp"] = pd.to_datetime(log_df["time:timestamp"], errors='coerce')

# Sort within case then compute offset from first event per case
log_df = log_df.sort_values([case_id_col, "time:timestamp"])
log_df['relative_time'] = (
    log_df.groupby(case_id_col)["time:timestamp"]
      .transform(lambda s: (s - s.min()).dt.total_seconds() / 60.0)
)

print("[rel-time] Created 'relative_time' (head):")
print(log_df[[case_id_col, "time:timestamp", 'relative_time']].head())
display(log_df.head(20))

[rel-time] Created 'relative_time' (head):
  SEHRegistratienummer      time:timestamp  relative_time
0           0000155083 2019-01-01 10:23:00            0.0
1           0000155083 2019-01-01 10:45:00           22.0
2           0000155083 2019-01-01 10:45:00           22.0
3           0000155083 2019-01-01 10:45:00           22.0
4           0000155083 2019-01-01 10:51:00           28.0


,SEHRegistratienummer,Activity,time:timestamp,Age,Gender,Specialismecode,Resource,relative_timestamp_from_previous_activity,relative_timestamp_from_start,label,relative_time
0,0000155083,registration,2019-01-01 10:23:00,86.0,Vrouw,ORT,nurse_D,0.0,0.0,Vrouw,0.0
1,0000155083,arrivel,2019-01-01 10:45:00,86.0,Vrouw,ORT,nurse_D,22.0,0.0,Vrouw,22.0
2,0000155083,Medicatieverificatie SEH,2019-01-01 10:45:00,86.0,Vrouw,ORT,doctor_D,0.0,0.0,Vrouw,22.0
3,0000155083,start treatment,2019-01-01 10:45:00,86.0,Vrouw,ORT,doctor_D,0.0,0.0,Vrouw,22.0
4,0000155083,triage,2019-01-01 10:51:00,86.0,Vrouw,ORT,nurse_D,6.0,0.0,Vrouw,28.0
5,0000155083,Beleidsorder SEH,2019-01-01 11:37:00,86.0,Vrouw,ORT,doctor_D,46.0,0.0,Vrouw,74.0
6,0000155083,Radiologie,2019-01-01 11:37:00,86.0,Vrouw,ORT,doctor_F,0.0,0.0,Vrouw,74.0
7,0000155083,Radiologieaanvraag SEH,2019-01-01 11:37:00,86.0,Vrouw,ORT,doctor_A,0.0,0.0,Vrouw,74.0
8,0000155083,SEH Radiologie bespreking,2019-01-01 11:37:00,86.0,Vrouw,ORT,doctor_D,0.0,0.0,Vrouw,74.0
9,0000155083,Radiologieaanvraag SEH,2019-01-01 12:39:00,86.0,Vrouw,ORT,doctor_A,62.0,0.0,Vrouw,136.0


## Save event log to CSV

This saves the current `log_df` to `output/event_log.csv` under the repository root.
- Columns are ordered with `[case_id, activity, timestamp]` first for convenience.
- The folder is created if it does not exist.

In [24]:
# Save event log
SAVE_DIR = base_root / 'data' / 'emergency_ORT'
SAVE_DIR.mkdir(parents=True, exist_ok=True)
SAVE_PATH = SAVE_DIR / 'emergency_ORT.csv'

# Rename case id column to 'Case ID' for output
out_df = log_df.copy()
if case_id_col in out_df.columns and case_id_col != 'Case ID':
    out_df.rename(columns={case_id_col: 'Case ID'}, inplace=True)

# Reorder columns: Case ID, activity, timestamp first
cols_front = ['Case ID', 'activity', 'timestamp']
cols_front = [c for c in cols_front if c in out_df.columns]
other_cols = [c for c in out_df.columns if c not in cols_front]
out_df = out_df.loc[:, cols_front + other_cols]

# Save with semicolon delimiter
out_df.to_csv(SAVE_PATH, index=False, sep=';')
print(f"Saved event log to: {SAVE_PATH}")
print(f"Rows: {len(out_df):,}, Columns: {len(out_df.columns)}")

Saved event log to: C:\Users\nikol\MT-repo\data\emergency_ORT\emergency_ORT.csv
Rows: 33,812, Columns: 11


### Split the saved event log into TRAIN/VAL/TEST (temporal)


In [25]:
# Split the saved event log
from src.split_log import split_temporal
from pathlib import Path
import pandas as pd

# Ensure we have a saved CSV path from the previous cell
if 'SAVE_PATH' not in globals():
    raise RuntimeError("SAVE_PATH not found — run the 'Save event log' cell first.")

# Derive output directory and base name
_csv_path = Path(SAVE_PATH)
_out_dir = _csv_path.parent  # same folder as saved log
_log_name = _csv_path.stem

# Workaround: create a temp CSV with a 'time:timestamp' alias so read_log parses datetimes
_tmp_path = _csv_path.with_name(f"{_log_name}__split_tmp.csv")
_df_tmp = pd.read_csv(_csv_path, sep=';')
if 'time:timestamp' not in _df_tmp.columns and 'timestamp' in _df_tmp.columns:
    _df_tmp['time:timestamp'] = _df_tmp['timestamp']
_df_tmp.to_csv(_tmp_path, index=False, sep=';')

# Split 60/20/20 by case start time, using our column names
split_temporal(
    log_path=str(_tmp_path),
    log_name=_log_name,
    output_path=str(_out_dir),
    split_perc=[0.6, 0.2, 0.2],
    csv_sep=';',
    case_id_key='Case ID',
    timestamp_key='time:timestamp',
)

print(f"Wrote TRAIN/VAL/TEST splits to: {_out_dir}")


Train cases: 1452, Val cases: 484, Test cases: 485
Wrote TRAIN/VAL/TEST splits to: C:\Users\nikol\MT-repo\data\emergency_ORT


## Quick cHSIC check on current log

This cell computes cHSIC(R, T | A) on the in-memory `log_df` using the same approach as the evaluation pipeline:
- For each activity, build a categorical kernel on Resource and a multi-scale RBF on relative time (minutes since case start).
- Use the unbiased HSIC estimator when feasible, else biased.
- Report per-activity HSICs and uniform/frequency-weighted aggregates.

Note: If you changed resource assignment just above, re-run that cell first, then run this one.

In [26]:
display(log_df.head(10))

,SEHRegistratienummer,Activity,time:timestamp,Age,Gender,Specialismecode,Resource,relative_timestamp_from_previous_activity,relative_timestamp_from_start,label,relative_time
0,0000155083,registration,2019-01-01 10:23:00,86.0,Vrouw,ORT,nurse_D,0.0,0.0,Vrouw,0.0
1,0000155083,arrivel,2019-01-01 10:45:00,86.0,Vrouw,ORT,nurse_D,22.0,0.0,Vrouw,22.0
2,0000155083,Medicatieverificatie SEH,2019-01-01 10:45:00,86.0,Vrouw,ORT,doctor_D,0.0,0.0,Vrouw,22.0
3,0000155083,start treatment,2019-01-01 10:45:00,86.0,Vrouw,ORT,doctor_D,0.0,0.0,Vrouw,22.0
4,0000155083,triage,2019-01-01 10:51:00,86.0,Vrouw,ORT,nurse_D,6.0,0.0,Vrouw,28.0
5,0000155083,Beleidsorder SEH,2019-01-01 11:37:00,86.0,Vrouw,ORT,doctor_D,46.0,0.0,Vrouw,74.0
6,0000155083,Radiologie,2019-01-01 11:37:00,86.0,Vrouw,ORT,doctor_F,0.0,0.0,Vrouw,74.0
7,0000155083,Radiologieaanvraag SEH,2019-01-01 11:37:00,86.0,Vrouw,ORT,doctor_A,0.0,0.0,Vrouw,74.0
8,0000155083,SEH Radiologie bespreking,2019-01-01 11:37:00,86.0,Vrouw,ORT,doctor_D,0.0,0.0,Vrouw,74.0
9,0000155083,Radiologieaanvraag SEH,2019-01-01 12:39:00,86.0,Vrouw,ORT,doctor_A,62.0,0.0,Vrouw,136.0


In [27]:
# cHSIC(R, T | A) — in-notebook check aligned with evaluation
import numpy as np
import pandas as pd
import torch

# 1) Prepare DF with standardized schema
_df = log_df.copy()

# Detect schema and unify column names
if {'case:concept:name', 'concept:name', 'org:resource'}.issubset(_df.columns):
    case_col, act_col, res_col = 'case:concept:name', 'concept:name', 'org:resource'
elif {'SEHRegistratienummer', 'Activity', 'Resource'}.issubset(_df.columns):
    _df = _df.rename(columns={'SEHRegistratienummer': 'case:concept:name','Activity': 'concept:name','Resource': 'org:resource'})
    case_col, act_col, res_col = 'case:concept:name', 'concept:name', 'org:resource'
else:
    raise ValueError("Missing required columns: expected either ['case:concept:name','concept:name','org:resource'] or ['Case ID','Activity','Resource']")

# If a clean 'Resource' column exists, prefer it to override any pre-existing 'org:resource'
if 'Resource' in _df.columns and _df['Resource'].notna().any():
    _df['org:resource'] = _df['Resource'].astype(str)

# Robustly coerce org:resource to scalar strings (guard against list/tuple/ndarray entries)
# Keep a helper that can stringify nested sequences deterministically

def _res_to_str(x):
    if isinstance(x, (list, tuple, np.ndarray)):
        try:
            return '|'.join(map(str, list(x)))
        except Exception:
            return str(x)
    return str(x)

# Apply once globally to catch typical cases
_df['org:resource'] = _df['org:resource'].apply(_res_to_str)

if 'time:timestamp' not in _df.columns:
    raise ValueError("Missing 'time:timestamp' column for cHSIC prep.")

# Ensure dtypes
_df['concept:name'] = _df['concept:name'].astype(str)
_df['org:resource'] = _df['org:resource'].astype(str)
_df['time:timestamp'] = pd.to_datetime(_df['time:timestamp'])

# 2) HSIC utilities — mirroring evaluation pipeline

def _rbf_multi(x: torch.Tensor, y: torch.Tensor, sigmas: torch.Tensor) -> torch.Tensor:
    d2 = torch.cdist(x, y, p=2).pow(2)
    K = 0.0
    for s in sigmas:
        K = K + torch.exp(-d2 / (2.0 * (s * s + 1e-12)))
    return K

def _delta_eq(x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    return (x[:, None] == y[None, :]).float()

def _hsic_unbiased(K: torch.Tensor, L: torch.Tensor) -> float:
    n = K.shape[0]
    if n < 4:
        return _hsic_biased(K, L)
    K2 = K.clone(); L2 = L.clone()
    K2.fill_diagonal_(0.0); L2.fill_diagonal_(0.0)
    H = torch.eye(n, device=K.device) - (1.0 / n) * torch.ones((n, n), device=K.device)
    KH = K2 @ H
    LH = L2 @ H
    val = (KH * LH).sum() / (n * (n - 3))
    return float(val.item())

def _hsic_biased(K: torch.Tensor, L: torch.Tensor) -> float:
    n = K.shape[0]
    H = torch.eye(n, device=K.device) - (1.0 / n) * torch.ones((n, n), device=K.device)
    Kc = H @ K @ H
    Lc = H @ L @ H
    val = (Kc * Lc).sum() / (n * n)
    return float(val.item())

def _median_bandwidth(v: torch.Tensor) -> float:
    n = v.shape[0]
    if n < 2:
        return 1.0
    exact_limit = 4000
    sample_size = 1024
    mad_fallback_limit = 20000
    if n <= exact_limit:
        d = torch.cdist(v, v, p=2)
        mask = d > 0.0
        med = torch.median(d[mask]) if torch.any(mask) else torch.tensor(1.0, device=v.device)
        return float(max(med.item(), 1e-3))
    if n > mad_fallback_limit:
        v_flat = v.view(-1)
        median_v = v_flat.median()
        mad = (v_flat - median_v).abs().median()
        approx = float((mad * 1.41421356237).item())
        return max(approx, 1e-3)
    with torch.no_grad():
        idx = torch.randperm(n, device=v.device)[:sample_size]
        vs = v[idx]
        d_sample = torch.cdist(vs, vs, p=2)
        mask = d_sample > 0.0
        med = torch.median(d_sample[mask]) if torch.any(mask) else torch.tensor(1.0, device=v.device)
    return float(max(med.item(), 1e-3))

# 3) Compute per-activity HSIC
_device = torch.device('cpu')
profile = {}
counts = _df['concept:name'].value_counts().to_dict()
max_points = 4000

for a, grp in _df.groupby('concept:name'):
    # Build a clean 1D string series for resources — defensive against 2D/nested entries
    vals = grp['org:resource'].to_numpy()
    if isinstance(vals, np.ndarray) and vals.ndim > 1:
        # Coerce each row (e.g., array([x,y])) to a single string 'x|y'
        g_res = pd.Series(['|'.join(map(str, list(v))) for v in vals], index=grp.index)
    else:
        g_res = grp['org:resource'].apply(_res_to_str)

    # Encode resource
    r_codes, _ = pd.factorize(g_res, sort=True)
    r = torch.from_numpy(np.asarray(r_codes, dtype=np.int64))

    # Time numeric
    t = torch.from_numpy(grp['relative_time'].values.astype(np.float32)).view(-1, 1)
    n = t.shape[0]
    if n < 3:
        continue
    # Subsample for memory safety
    if n > max_points:
        idx = torch.randperm(n)[:max_points]
        t = t[idx]; r = r[idx]
        n = max_points
    # Kernels
    s = _median_bandwidth(t)
    sigmas = torch.tensor([s, 2*s, 4*s], dtype=torch.float32)
    Kt = _rbf_multi(t, t, sigmas)
    Kr = _delta_eq(r, r)
    # HSIC
    hsic_val = _hsic_unbiased(Kt, Kr) if n >= 4 else _hsic_biased(Kt, Kr)
    profile[str(a)] = hsic_val

# 4) Aggregates
if profile:
    uniform = float(np.mean(list(profile.values())))
    total = 0.0; acc = 0.0
    for a, v in profile.items():
        w = counts.get(a, 0)
        acc += w * v
        total += w
    freq_weighted = float(acc / total) if total > 0 else None
else:
    uniform = None; freq_weighted = None

print("cHSIC per activity (first 10):")
for k, v in list(sorted(profile.items(), key=lambda kv: kv[1], reverse=True))[:10]:
    print(f"  {k}: {v:.6f}")
print(f"\nAggregate cHSIC — uniform: {uniform}, freq-weighted: {freq_weighted}")

# Optional: keep a small dict to inspect
chsic_result = {
    'per_activity': profile,
    'aggregate': {'uniform': uniform, 'freq_weighted': freq_weighted},
    'meta': {'num_events': int(len(_df))}
}

cHSIC per activity (first 10):
  Beleid klinisch: 0.004432
  Pijnscore herhalen: 0.002944
  Inspectie insteekopening infuus: 0.002495
  Externe verslaglegging: 0.002396
  Preoperatieve checklist + opname stopmoment: 0.002263
  Ontslagbeleid Poliklinisch - SEH: 0.002077
  Aanvraag opname: 0.002013
  depart: 0.001684
  Opname: 0.001671
  Afspraak maken (over 1 week): 0.001665

Aggregate cHSIC — uniform: 0.00092353800489166, freq-weighted: 0.0009050566349068657
